In [ ]:
# 

from bs4 import BeautifulSoup  # For parsing HTML content
import requests  # For sending HTTP requests to the URL
import pandas as pd
import re # to thelw gia ti sinartisi pou allazei selida 

# The URL of the website from which we want to scrape data
url = input('vale to url')
url = url + 'page/1/'
# print(url)

excel_file_name = input('vale to onoma tou excel arxeiou pou theleis na apothikeytoun')

# Sending a GET request to the provided URL and fetching the page content
page = requests.get(url)
# Parsing the page content using BeautifulSoup with lxml parser
soup = BeautifulSoup(page.content, 'lxml')

buttons_container = soup.find('div' , class_ = 'm-ghostblock')
# print(buttons_container)
pages_nr = buttons_container.find_all('a' , class_ = "m-ghostblock__nav-item")
# print(len(pages_nr))

# h synartisi auti allazei selida
def increment_page_number(local_url):
    global url
    # Use a regular expression to find the page number pattern "/number/"
    match = re.search(r'\/(\d+)\/$', local_url)
    if match:
        # Get the current page number
        current_page = int(match.group(1))
        # Increment the page number
        new_page = current_page + 1
        # Replace the old page number with the new one in the URL
        url = re.sub(r'\/(\d+)\/$', f'/{new_page}/', local_url)
        return url
    else:
        # If there's no page number, return the original URL
        print("No page number found in the URL.")
        
tools_url_list = []
def collecting_urls_fun(local_url):
    global tools_url_list
    # Sending a GET request to the provided URL and fetching the page content
    page = requests.get(url)
    # Parsing the page content using BeautifulSoup with lxml parser
    soup = BeautifulSoup(page.content, 'lxml')
    # Find the section containing the product grid (the tools)
    tools_container = soup.find('div', class_='col-xs-12 col-lg-9')
    # Find all individual product tiles in the grid
    tools_list = tools_container.find_all('div', class_='category-grid-tile a-category-grid-tile')
    # Loop through each product and extract the necessary data
    for tool in tools_list:
        # Extract the product URL
        product_url = tool.find('a', class_='category-grid-tile__link-wrapper').get('href')
        tools_url_list.append(product_url)


tools_container = soup.find('div' , class_ = 'col-xs-12 col-lg-9')
if tools_container == None:
    collecting_urls_fun
    print('there is no container')
else:
    tools_list = tools_container.find_all('div' , class_ = 'category-grid-tile a-category-grid-tile')
    if len(tools_list) < 2 :
        collecting_urls_fun(url)
        print('there is no button')
    else:
        collecting_urls_fun(url)
        # kalw loipon tin sullogi url gia prwti fora. 
        # pleon ksekinaw ti loupa. 
        for i in range(len(pages_nr)):
            increment_page_number(url)
            collecting_urls_fun(url)

print(tools_url_list)
    

# List to hold all data
all_data = []

#  list for Σετ εργαλειων
sets_ergaleiwn = []

# Function to extract technical specifications and product code from the provided URL
def tool_page_funct(url):
    global all_data  # Use the global list to store data across function calls
    global sets_ergaleiwn

    # Sending a GET request to the provided URL and fetching the page content
    page = requests.get(url)

    # Parsing the page content using BeautifulSoup with lxml parser
    soup = BeautifulSoup(page.content, 'lxml')

    # check if the tool is a set: 
    set_detector = soup.find('p' , class_ = 'product-detail-stage__subtitle headline__redesign headline__redesign--blue')
    set_detector = set_detector.get_text()
    set_detector_2 = set_detector.find('Σετ')
    print('the set detector is ' , set_detector)
    print('the set_detector_2 is ' , set_detector_2)


    if set_detector_2 == -1 :

        try:

            # Find the section containing the product details
            entry_specs = soup.find("div", class_="product-detail-stage__info")

            print('the url is ' , url)
            # Find the headline, if needed
            praising = entry_specs.find('h3', class_='product-detail-stage__headline')
            praising = praising.get_text() if praising else "No headline"

            # Find the list of specifications
            entry_specs_list = entry_specs.find('ul', class_='product-detail-stage__list a-list')
            # Find all the <li> tags
            li_tags = entry_specs_list.find_all('li')

            # Extract the text from each <li> tag and join them with '\n'
            entry_specs = "\n".join([li.get_text().strip() for li in li_tags])
            entry = f"{praising}\n{entry_specs}"
            entry = [entry]  # Store the entry in a list

            #  safety guard for epilogue paragraph : 
        except:
            entry = ['None']

        try: 

            epilogue_container = soup.find('div', class_='col-md-10')
            # print(epilogue_container)
            epilogue_headers = epilogue_container.find_all('h3', class_='bba--problue bba--fw-bold bba--big m-a-product_highlights__space')
            # Extracting the text from each h3 tag
            epilogue_headers = [feature.get_text() for feature in epilogue_headers]

            epilogue_praising = epilogue_container.find_all('p')
            epilogue_praising = [feature.get_text() for feature in epilogue_praising]

            if len(epilogue_headers) != len(epilogue_praising):
                print('There is a problem in the epilogue. The number of the headers != the number of the text (epilogue_praising)')
            else:
                # Now, combine headers and corresponding paragraphs into one list
                combined_dumy_list = []

                # Ensure you pair the headers with the paragraphs (assumes same number of headers and paragraphs)
                for header, para in zip(epilogue_headers, epilogue_praising):
                    combined_dumy_list.append(f"{header}\n{para}")

                # Combine all pairs into a single string, with each header-paragraph pair separated by a newline
                epilogue_specs = "\n".join(combined_dumy_list)

                # Place the final output into a list as per your request
                epilogue_specs = [epilogue_specs]
        except:
            print('πιθανοτατα δεν εχει καθολου παραγραφο επιλογου.')
            epilogue_specs =['None']

        # Check if variations_box exists to prevent errors
        variations_box = soup.find('div', class_='col-sm-12 o-b-product_variations__col')

        if variations_box:
            variations = variations_box.find_all("div", class_="col-sm-4")

            # Create lists to store part numbers, variations, items included for each variation, and technical specifications
            part_numbers = []
            variations_list = []
            items_included_list = []

            for container in variations:
                # Extracting the description (variation name)
                description = container.find("div", class_="headline headline__redesign headline__redesign--blue headline__redesign--hl5 o-b-product_variations__productHeadline")
                description = description.get_text() if description else "No description"

                # Extracting the part number (not used in the combined info column)
                part_nr = container.find("p", class_="o-b-product_variations__partNr")
                part_nr = part_nr.find('span')
                part_nr = part_nr.get_text() if part_nr else "No part number"

                # Extracting the items included in the package
                including_container = container.find_all("ul", class_='a-list')
                including_list_final = []
                for including_list in including_container:
                    periexomena = including_list.find_all('li')
                    item_included = [li.get_text().strip() for li in periexomena]
                    including_list_final.append(item_included)

                # Flatten the nested list
                flattened_list = [item for sublist in including_list_final for item in sublist]

                # Remove unnecessary newlines and extra spaces
                flattened_list = [item.strip() for item in flattened_list]

                # Append the description (variation), part number, and items included
                part_numbers.append(part_nr)
                variations_list.append(description)
                items_included_list.append("\n".join(flattened_list))

            # Extract the technical specifications only once (since it's the same for all)
            technical_specs = soup.find('div', class_='col-lg-8 col-xs-12')
            if technical_specs:
                tables = technical_specs.find_all("table")
                lista = []
                for table in tables:
                    tbody = table.find('tbody')
                    line = tbody.find_all("td")
                    for tr in line:
                        text_of_line = tr.get_text().strip()
                        lista.append(text_of_line)

                # Clean the extracted data by removing newline characters
                cleaned_data = [item.replace('\n', '') for item in lista]
                # Join all elements with a newline separator
                combined_technical_specs = "\n".join(cleaned_data)
            else:
                combined_technical_specs = "No technical specifications available."

            # Add the technical specifications to all rows
            technical_specs_list = [combined_technical_specs] * len(part_numbers)

            # Add the epilogue_specs to each row to match the length of part_numbers
            epilogue_specs_list = epilogue_specs * len(part_numbers)  # Replicate for each variation

            # Combine everything into a single list of dictionaries
            for i in range(len(part_numbers)):
                all_data.append({
                    "Part Number": part_numbers[i],
                    "Variation": variations_list[i],
                    "Items Included in the Variation": items_included_list[i],
                    "Technical Specifications": technical_specs_list[i],
                    "Entry Level Specs": entry[0],  # Add entry level specs once
                    "Epilogue Specs": epilogue_specs_list[i]  # Add epilogue specs for each variation
                })

            print("Data collected for this product.")
        else:
            print("Could not find the variations section.")

    else:
        print('einai set ergaleiwn pithanotata. apothikeuetai sti lista ')
        sets_ergaleiwn.append(url)
# # Call the function for the first time
# tool_page_funct(url)


for p_url in tools_url_list:
    tool_page_funct(p_url)

excel_file_name = excel_file_name + '.xlsx'

# Convert the collected data into a DataFrame
df = pd.DataFrame(all_data)

# Save the data to Excel once the scraping is complete
df.to_excel(excel_file_name , index=False)
print(f"Data saved to {excel_file_name}")

['https://www.bosch-professional.com/gr/el/products/grw-18v-120-06011C2000', 'https://www.bosch-professional.com/gr/el/products/grw-18v-160-06011C3000', 'https://www.bosch-professional.com/gr/el/products/grw-12-e-06011A7000', 'https://www.bosch-professional.com/gr/el/products/grw-18-2-e-06011A8000', 'https://www.bosch-professional.com/gr/el/products/grw-140-06011C4020']
the set detector is  Αναδευτήρας
the set_detector_2 is  -1
the url is  https://www.bosch-professional.com/gr/el/products/grw-18v-120-06011C2000
Data collected for this product.
the set detector is  Αναδευτήρας
the set_detector_2 is  -1
the url is  https://www.bosch-professional.com/gr/el/products/grw-18v-160-06011C3000
Data collected for this product.
the set detector is  Αναδευτήρας
the set_detector_2 is  -1
the url is  https://www.bosch-professional.com/gr/el/products/grw-12-e-06011A7000
Data collected for this product.
the set detector is  Αναδευτήρας
the set_detector_2 is  -1
the url is  https://www.bosch-profession